In [1]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [3]:
# 读取数据
# MNIST数据集是手写数字数据库，包括60,000个示例的训练集和一个包含10,000个示例的测试集，每一个示例是一张28*28像素的手写数字图像（单一通道）
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
# 设置参数和占位符，并将数据集图像转化为张量
width = 28 #以像素为单位的图像宽度
height = 28 #以像素为单位的图像高度
flat = width * height # 一个图像中的像素数
class_output = 10 # 问题可能分类的数量

x  = tf.placeholder(tf.float32, shape=[None, flat])
y_ = tf.placeholder(tf.float32, shape=[None, class_output])

# 输入图像是28像素×28像素，1个通道（灰度）
# 第一个维度是图像的批次编号，可以是任意大小（将其设置为-1）
# 第二和第三个维度是宽度和高度，最后一个是图像通道
x_image = tf.reshape(x, [-1,28,28,1])  

In [6]:
# 设置卷积层1
# 这里过滤器内核大小是5*5; 输入通道为1（灰度）
# 需要32个不同的特征映射（这里，32个特征映射表示32个不同的滤波器应用在每个图像上，因此卷积层的输出将是28*28*32）
W_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1))
b_conv1 = tf.Variable(tf.constant(0.1, shape=[32])) # need 32 biases for 32 outputs
convolve1= tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1
# ReLU激励函数
# ReLU激励函数的作用是，将covolve1中负数出现的任何地方，用0替换
h_conv1 = tf.nn.relu(convolve1)
# 最大池
# 此处最大池的内核为2*2，接移动步长为2，即将内核做滑动 窗口作用于图像，且取窗口内最大值。此处达到了降维的效果，最后输出为14*14*32的矩阵
conv1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [7]:
# 设置卷积层2
# 第一到第二层，输入的是[14x14x32]的图像，过滤器是[5x5x32]的内核
# 使用尺寸[5x5x32]的64级的过滤器，即取了64个特征
W_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64])) #需要64输出的64个偏差
convolve2= tf.nn.conv2d(conv1, W_conv2, strides=[1, 1, 1, 1], padding='SAME')+ b_conv2
# ReLU激励函数
h_conv2 = tf.nn.relu(convolve2)
# 最大池
# 最后通过最大池，第二层输出的是[7x7x64]
conv2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 

In [8]:
# 设置第三层完全连接
# 完全连接的图层才能使用Softmax并最终创建概率
# 完全连接的图层从上一层获取高级过滤的图像，即所有64个图形，并将它们转换为平面数组
# 因此，每个矩阵[7x7]将被转换为[49x1]的矩阵，然后所有的64矩阵都将被连接起来，形成一个大小为[3136x1]的数组
# 将它连接到另一个尺寸为[1024x1]的图层。所以，这两层之间的重量将是[3136x1024]
layer2_matrix = tf.reshape(conv2, [-1, 7*7*64])
W_fc1 = tf.Variable(tf.truncated_normal([7 * 7 * 64, 1024], stddev=0.1))
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024])) 
fcl=tf.matmul(layer2_matrix, W_fc1) + b_fc1# need 1024 biases for 1024 outputs

h_fc1 = tf.nn.relu(fcl)#ReLU激励函数

In [9]:
# 设置丢弃层，防止过拟合
keep_prob = tf.placeholder(tf.float32)
layer_drop = tf.nn.dropout(h_fc1, keep_prob)

In [10]:
# 设置输出层（Softmax层）
W_fc2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1)) #1024个神经元
b_fc2 = tf.Variable(tf.constant(0.1, shape=[10])) 
fc=tf.matmul(layer_drop, W_fc2) + b_fc2
y_CNN= tf.nn.softmax(fc)# 10 possibilities for digits [0,1,2,3,4,5,6,7,8,9]

In [11]:
# 训练函数
# 定义损失函数、优化器
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_CNN), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_CNN,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [12]:
# 运行会话，开始训练
sess.run(tf.global_variables_initializer())
# 进行1100次迭代
for i in range(1100):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, float(train_accuracy)))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

print("test accuracy %g"%accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.02
step 100, training accuracy 0.74
step 200, training accuracy 0.96
step 300, training accuracy 0.96
step 400, training accuracy 0.98
step 500, training accuracy 0.96
step 600, training accuracy 0.98
step 700, training accuracy 0.9
step 800, training accuracy 1
step 900, training accuracy 0.94
step 1000, training accuracy 0.9
test accuracy 0.9646
